In [2]:
import re
import json
import csv
import pandas as pd
from collections import Counter

In [1]:
def get_lemma_by_term(query_term:str, lang:str) -> str:
    '''
    Getting a lemma of a query term
    lang: str, 'en' or 'nl'
    Returns str, 'not found' if lemma was not found
    '''
    
    return_lemma = 'not found'
    
    # importing query terms with lemmas
    # change path to GitHub
    
    with open('/Users/anesterov/reps/LODlit/query_terms.json','r') as jf:
        query_terms = json.load(jf)
        
    for lemma, qt in query_terms[lang].items():
        if query_term in qt:
            return_lemma = lemma
            
    return return_lemma

## Wikidata

In [3]:
def search_implicit_markers_wd(lang:str):
    '''
    lang: str, "en" or "nl"
    Returns pandas DataFrame
    '''
    
    wd_implicit = pd.DataFrame(columns=["resource",lang,"lemma","entity_id","property","value","implicit_marker","level"])
    
    # importing implicit markers dict
    with open('/Users/anesterov/reps/LODlit/sensitivity_markers/implicit/implicit_markers.json','r') as jf:
        implicit_markers = json.load(jf)
    
    # importing all search results
    with open(f"/Users/anesterov/LODlit_local/wd/jan31/results_clean_{lang}.json",'r') as jf:
        wd_all = json.load(jf)
        
    # import subset
    with open(f"/Users/anesterov/reps/LODlit/Wikidata/wd_{lang}_subset.json",'r') as jf:
        wd_subset = json.load(jf)
        
    # get all QIDs in the subset
    subset_quids = []
    for hits in wd_subset.values():
        for hit in hits:
            subset_quids.append(hit["QID"])
            
    # import rm
    wd_rm = pd.read_csv("/Users/anesterov/reps/LODlit/rm/rm_entities_unique.csv")
    rm_quids = list(wd_rm[wd_rm["resource"] == "wikidata"][wd_rm["lang"] == lang]["entity_id"])
    
    # searching in descriptions
    for term, hits in wd_all.items():
        lemma = get_lemma_by_term(term, lang)
        
        for hit in hits:

            level = "1"
            # check entity level
            if hit["QID"] in set(subset_quids):
                level = "2"
            if hit["QID"] in set(rm_quids):
                level = "3"

            # check descriptions type
            if type(hit["description"]) == list:
                for d in hit["description"]:
                    # iterating over all markers
                    for marker in implicit_markers["wikidata"][lang]:
                        match = re.search(f"\\b{marker}\\b",d)
                        if match:
                            row = ["wikidata",lang,lemma,hit["QID"],"description",d,match[0],level]
                            wd_implicit.loc[len(wd_implicit)] = row

            if type(hit["description"]) == str:
                for marker in implicit_markers["wikidata"][lang]:
                    match = re.search(f"\\b{marker}\\b",hit["description"])
                    if match:
                        row = ["wikidata",lang,lemma,hit["QID"],"description",hit["description"],match[0],level]
                        wd_implicit.loc[len(wd_implicit)] = row
                        
    return wd_implicit.drop_duplicates(ignore_index=True)

In [7]:
wd_nl_implicit = search_implicit_markers_wd("nl")

<ipython-input-3-93c9a52dc902>:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rm_quids = list(wd_rm[wd_rm["resource"]=="wikidata"][wd_rm["lang"]==lang]["entity_id"])


In [8]:
# export csv
wd_nl_implicit.to_csv("wd_nl_implicit.csv")

## AAT